<b>Section 1: Setup & Import</b>

In [4]:
# 🚀 Always run this cell first! It contains all necessary imports.

import gc
import sys
import pprint

import requests
import json

import numpy as
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

SyntaxError: invalid syntax (2190478764.py, line 10)

In [2]:
# NHL API URL for team stats
url = "https://api.nhle.com/stats/rest/en/team"

# Fetch data from the API
response = requests.get(url)

# Convert response to JSON format
data = response.json()

# Extract relevant information
#teams = []
#for team in data["teams"]:
#    teams.append({
#        "id": team["id"],
#        "name": team["name"],
#      "abbreviation": team["abbreviation"],
#        "venue": team["venue"]["name"],
#        "city": team["venue"]["city"],
#        "division": team["division"]["name"],
#        "conference": team["conference"]["name"],
#        "first_year_of_play": team["firstYearOfPlay"]
#    })

# Extract the correct key (if "data" contains the list)
if "data" in data:
    df_teams = pd.DataFrame(data["data"])
else:
    df_teams = pd.DataFrame()  # Create an empty DataFrame if no data found


# Display DataFrame info
print(df_teams.info())  # Check if data is loaded correctly
print(df_teams.head())  # Display first few rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           61 non-null     int64  
 1   franchiseId  59 non-null     float64
 2   fullName     61 non-null     object 
 3   leagueId     61 non-null     int64  
 4   rawTricode   61 non-null     object 
 5   triCode      61 non-null     object 
dtypes: float64(1), int64(2), object(3)
memory usage: 3.0+ KB
None
   id  franchiseId              fullName  leagueId rawTricode triCode
0  32         27.0      Quebec Nordiques       133        QUE     QUE
1   8          1.0    Montréal Canadiens       133        MTL     MTL
2  58          5.0  Toronto St. Patricks       133        TSP     TSP
3   7         19.0        Buffalo Sabres       133        BUF     BUF
4  46         13.0         Oakland Seals       133        OAK     OAK


In [3]:
show_memory_usage()
optimize_dataframe(df_teams)

NameError: name 'show_memory_usage' is not defined

In [ ]:
print(df_teams.isnull().sum())  # check for missing values

In [ ]:
df_teams["franchiseId"].fillna(-1, inplace=True)  # Replace NaN with -1

In [ ]:
print(df_teams.isnull().sum())  # check for missing values

In [ ]:
df_teams.rename(columns={"fullName": "team_name", "triCode": "team_code"}, inplace=True)  #rename columns for readability


In [ ]:
#visualize number of teams by league

plt.figure(figsize=(8, 4))
sns.countplot(x=df_teams["leagueId"])
plt.title("Number of Teams by League")
plt.xlabel("League ID")
plt.ylabel("Team Count")
plt.show()

In [ ]:
df_teams["leagueId"].value_counts()  #list + counts of League IDs 

In [ ]:
#visualize number of teams by team code
plt.figure(figsize=(8, 4))
sns.countplot(x=df_teams["team_code"])
plt.title("Number of Teams by Team Code")
plt.xlabel("Team Code")
plt.ylabel("Team Count")
plt.show()

In [ ]:
df_teams["team_code"].value_counts()  #list + counts of team IDs 

In [ ]:
# fetch list of players on roster for 2024-2025 season (first attempt)

# NHL API URL for player list
url = "https://api-web.nhle.com/v1/roster/TOR/20242025"  # Adjust for the season and team name (Toronto, 2024-25 season)

# Fetch data from API
response = requests.get(url)
data = response.json()

# Convert response to DataFrame -- need to specify a KEY = goalies, forwards, defence; how the API response data is organized
df_players = pd.DataFrame(data["goalies"])

# Display first few rows
df_players.head()

In [ ]:
show_memory_usage()
optimize_dataframe(df_players)

In [ ]:
# fetch list of players on roster for 2024-2025 season (second attempt - get the keys/categories dynamically)

# NHL API URL for player list
url = "https://api-web.nhle.com/v1/roster/TOR/20242025"  # Adjust for the season and team name

# Fetch data from API
response = requests.get(url)
data = response.json()

#print top-level keys of dataset (as data is organized in forwards, goalies and defence; "key" or category = first level of nested JSON
for category in data.keys():
    print("Category:", category)

#print number of players by category/key and the first few players info
for category, players in data.items():
    print(f"\nCategory: {category} - {len(players)} players")

    # Print first few players in this category
    for player in players[:3]:  # Show first 3 players only
        print(player["id"], "-", player["firstName"], player["lastName"])

# Convert response to DataFrame
#df_players = pd.DataFrame(data["goalies"])

# Display first few rows
#df_players.head()

In [ ]:
# fetch list of players on roster for 2024-2025 season (third attempt - inspect the column names in player details/roster

# Fetch roster data
url = "https://api-web.nhle.com/v1/roster/TOR/20242025"  #specify team and season
response = requests.get(url)
data = response.json()

# Print full details of the first player (for example, from Forwards)
first_category = list(data.keys())[0]  # Get the first category (e.g., "Forwards")
first_player = data[first_category][0]  # Get the first player in that category

# Pretty-print the player data
print(json.dumps(first_player, indent=4))

In [ ]:
# capture all the player details automatically and put all the players into one flat list / dataframe 

all_players = []

# Loop through each category (Forwards, Goalies, Defense)
for category, players in data.items():
    for player in players:
        player_dict = player.copy()  # Copy player data
        player_dict["position"] = category  # Add category as position
        all_players.append(player_dict)

# Convert to DataFrame
df_players = pd.DataFrame(all_players)

# Display first few rows
df_players.head()

In [ ]:
show_memory_usage()
optimize_dataframe(df_players)

In [ ]:
#check available columns

print(df_players.columns)

In [ ]:
Now we will create a consolidated list of players from all teams in the current season
Overview of Steps

1️⃣ Get the list of team codes (from the Teams API)
2️⃣ Loop through each team and request their roster
3️⃣ Flatten the roster data (Forwards, Goalies, Defense)
4️⃣ Merge all team rosters into a single DataFrame


In [ ]:
#step 1 fetch list of team codes --> check keys

# NHL API URL to get team list
url = "https://api-web.nhle.com/v1/standings/now"  # API for current season standings (includes teams) - now = date
response = requests.get(url)
data = response.json()

# Extract team codes
teams = []

# see if there are keys
print("Top-level keys:", data.keys())


In [ ]:
#get the 3rd key = standings and check columns
first_category = list(data.keys())[2]
first_team = data[first_category][0]

print(json.dumps(first_team, indent=4))

In [ ]:
# STEP 1 retrieve all team codes and names from the API (now that we know what team fields to grab from standings)

# NHL API URL to get team list
url = "https://api-web.nhle.com/v1/standings/now"  # API for current season standings (includes teams)
response = requests.get(url)
data = response.json()

# Extract team codes
teams = []

for team in data["standings"]:
    teams.append({
        "teamAbbrev": team["teamAbbrev"],  # Short code (e.g., "PIT" for Pittsburgh Penguins)
        "teamName": team["teamName"],  # Full team name
        #"teamId": team["teamId"]  # Unique team ID
    })

# Convert to DataFrame
df_teams = pd.DataFrame(teams)

# Display first few rows
df_teams.head()

In [ ]:
show_memory_usage()
optimize_dataframe(df_teams)

In [ ]:
# STEP 2 loop through the teams and fetch roster

all_players = []  # List to store all player data

# Loop through each team and fetch their roster
for index, team in df_teams.iterrows():
    #team_id = team["teamId"]  # Get team ID
    team_name = team["teamName"]["default"]  # only extract the default value - it has a nested dictionary of format (default, value)
    team_abbrev = team["teamAbbrev"]["default"]
    
    print(f"Fetching roster for ({team_abbrev})...")

    # NHL API URL for team rosters
    roster_url = f"https://api-web.nhle.com/v1/roster/{team_abbrev}/20242025"
    
    try:
        response = requests.get(roster_url)
        data = response.json()
        
        # Loop through each category (Forwards, Goalies, Defense)
        for category, players in data.items():
            for player in players:
                player_dict = player.copy()  # Copy player data
                player_dict["position"] = category  # Add category (Forwards, Goalies, Defense)
                player_dict["team"] = team_name  # Add team name
                player_dict["teamAbbrev"] = team_abbrev  # Add team abbreviation
                
                all_players.append(player_dict)

    except Exception as e:
        print(f"Error fetching roster for {team_name}: {e}")

# Convert to DataFrame
df_players = pd.DataFrame(all_players)

# Display first few rows
df_players.head()


In [ ]:
show_memory_usage()
delete_dataframe(team)
optimize_dataframe(df_players)

In [ ]:
show_memory_usage()

In [ ]:
# Step 3 check and clean the data in consolidated player list - profile of data

df_players.info()  # Check data structure
df_players.columns  # List all columns
df_players.head()  # Preview data




In [ ]:
# check for missing values

df_players.isnull().sum()  # check for missing values


In [ ]:
df_players.drop(columns=["birthStateProvince"], inplace=True)  # drop column with too many missing values
df_players.columns


In [ ]:
#check for duplicates --> error because many fields (of type object) are dictionaries with (key, value) pairs - default then other values by language
df_players.duplicated().sum()

In [ ]:
# convert text fields to lowercase for consistency
df_players["team"] = df_players["team"].str.title()  # Capitalizes first letter
df_players["position"] = df_players["position"].str.upper()  # Convert to uppercase

In [ ]:
# convert height to consistent format --> error as there is no height column
df_players["height"] = df_players["height"].str.replace(" ft ", "'").str.replace(" in", "")

In [ ]:
# convert date to DateTime Format

df_players["birthDate"] = pd.to_datetime(df_players["birthDate"])

In [ ]:
# final check of data
df_players.info()
df_players.head()

In [ ]:
# save the cleaned data to CSV
df_players.to_csv("nhl_players_cleaned.csv", index=False)

In [ ]:
# visualize player data - plot # of players by position

plt.figure(figsize=(8, 5))
sns.countplot(y=df_players["position"], order=df_players["position"].value_counts().index, palette="coolwarm")

plt.title("Number of Players by Position", fontsize=14)
plt.xlabel("Count")
plt.ylabel("Position")
plt.show()

In [ ]:
# visualize player height and weight distribution

# Plot height distribution
plt.figure(figsize=(8, 5))
sns.histplot(df_players["heightInInches"].dropna(), bins=20, kde=True, color="blue")
plt.title("Distribution of Player Heights")
plt.xlabel("Height (inches)")
plt.ylabel("Count")
plt.show()

In [ ]:
# weight distribution

plt.figure(figsize=(8, 5))
sns.histplot(df_players["weightInPounds"].dropna(), bins=20, kde=True, color="red")
plt.title("Distribution of Player Weights")
plt.xlabel("Weight (lbs)")
plt.ylabel("Count")
plt.show()

In [ ]:
# height by position

plt.figure(figsize=(10, 6))
sns.boxplot(x=df_players["position"], y=df_players["heightInInches"], palette="muted")
plt.title("Player Height by Position")
plt.xlabel("Position")
plt.ylabel("Height (inches)")
plt.show()

In [ ]:
# weight by position

plt.figure(figsize=(10, 6))
sns.boxplot(x=df_players["position"], y=df_players["weightInPounds"], palette="muted")
plt.title("Player Weight by Position")
plt.xlabel("Position")
plt.ylabel("Weight (lbs)")
plt.show()

In [ ]:
#top 10 heaviest players

df_heaviest = df_players.sort_values(by="weightInPounds", ascending=False).head(10)

#extract lastName from dictionary (key,value pairs for different languages in the dataset), and convest to string
df_players["lastName"] = df_players["lastName"].apply(lambda x: x["default"] if isinstance(x, dict) else x)

plt.figure(figsize=(10, 5))
sns.barplot(y=df_heaviest["lastName"], x=df_heaviest["weightInPounds"], palette="magma")
plt.title("Top 10 Heaviest NHL Players")
plt.xlabel("Weight (lbs)")
plt.ylabel("Player Name")
plt.show()

In [ ]:
Next step - integrate player data with game stats so we can see how they are performing

Step 1 fetch NHL game results
Step 2 extract key game details
Step 3 merge player data with game data
Step 4 analyze and visualize

In [ ]:
#step 1 fetch NHL game results --> check the scoreboard API - has list of games by team, with scores... includes like to "gamecentre" with more stats

# NHL API URL for the game schedule/results for current season
season = "now"  # Update to current season
teamCode = "tor"
url = f"https://api-web.nhle.com/v1/scoreboard/{teamCode}/{season}"

# Fetch game data
response = requests.get(url)
data = response.json()

#check JSON structure of API response to know what fields to grab
print(json.dumps(data, indent=4))

"""
# Convert to DataFrame (updated post checking keys)
df_games = pd.DataFrame(data["gamesByDate"]["games"])

# Display first few rows
df_games.head()
"""

In [ ]:
#step 1b fetch NHL game results --> check the gamecenter API; every game has more details on player stats; we'll leave this alone for now

# Look up gamecenter game stats (detail) for a specific game - tor-vs-van; 2025-02-08; game ID: 2024020884 
gameID = 2024020884
url = f"https://api-web.nhle.com/v1/gamecenter/{gameID}/boxscore"

# Fetch game data
response = requests.get(url)
data = response.json()

#check JSON structure of API response to know what fields to grab
print(json.dumps(data, indent=4))

"""
# Convert to DataFrame (updated post checking keys)
df_games = pd.DataFrame(data["gamesByDate"]["games"])

# Display first few rows
df_games.head()
"""

In [ ]:
game_url = "https://api-web.nhle.com/v1/score/now"
response = requests.get(game_url)
data = response.json()
print(json.dumps(data, indent=4))

In [ ]:
#step 1 & 2 fetch NHL game results and extract key 
# this function uses the scoreboard API - which includes games 21 days ago and 16 games into future

def load_gamesFromScoreboardAPI(APIRequestDate):
    all_games = []  # List to store all games data
    
    # Loop through each team and fetch their games in the current season
    for index, team in df_teams.iterrows():
        team_name = team["teamName"]["default"]  # only extract the default value - it has a nested dictionary of format (default, value)
        team_abbrev = team["teamAbbrev"]["default"]
        
        print(f"Fetching games for ({team_abbrev})...")
        
        # NHL API URL for games this season for that team
        game_url = f"https://api-web.nhle.com/v1/scoreboard/{team_abbrev}/{APIRequestDate}"

        print(f"Fetching data from: {game_url}")
        
        try:
            #get API call - fetch game data
            response = requests.get(game_url)
            if response.status_code != 200:
                print(f"⚠️ Error: Received status code {response.status_code} from API")
                print(f"Response: {response.text}")  # Print error details
                continue  # Skip to next team if the request fails
            
            # retrieve JSON response
            data = response.json()

            # check if there's any data for this team
            if "gamesByDate" not in data or not data["gamesByDate"]:
                print(f"⚠️ No games found for {team_abbrev} on {APIRequestDate}")
                continue  # Skip to the next team
                
            # Loop through the JSON structure and pull out key game details (Game Date for that Team > Game > Key Fields) 
            for game_date in data["gamesByDate"]:
                            
                for game in game_date["games"]:
                    all_games.append({
                        "gameID": game["id"],
                        "gameDate": game["gameDate"],
                        "season": game["season"],
                        "gameDetailURL": game["gameCenterLink"],
                        "venue": game["venue"]["default"],
                        "startTime": game["startTimeUTC"],
                        "awayTeam": game["awayTeam"]["name"]["default"],
                        "awayTeamAbbrev": game["awayTeam"]["abbrev"],
                        "awayTeamScore": game.get("awayTeam",{}).get("score",0), # if score doesn't exist (game in future), then return 0 (for simplicity)
                        "homeTeam": game["homeTeam"]["name"]["default"],
                        "homeTeamAbbrev": game["homeTeam"]["abbrev"],
                        "homeTeamScore": game.get("homeTeam",{}).get("score",0) # if score doesn't exist (game in future), then return 0 (for simplicity)
                    })
            
        except Exception as e:
            print(f"Error fetching roster for {team_name}: {e}")  #error handling on API call
    
    
    # Convert to DataFrame
    df_games = pd.DataFrame(all_games)
    
    # Display first few rows
    df_games.info()
    df_games.head()   

    return df_games

df_games = load_gamesFromScoreboardAPI("now")


In [ ]:
## the df_games only has 344 entries - low for an NHL season!  Lets profile what the game dates are
df_games["gameDate"].value_counts()

In [ ]:
# by playing the API, I noticed it converts "now" to the current date.  You can change this to another date
# from the above output, it looks like the API provides a range of game dates
#lets calculate what this range is so we know how to expand the API search for the season

import datetime

def check_gameDateRangeInAPI(df):
    # Ensure gameDate is in datetime format
    df["gameDate"] = pd.to_datetime(df["gameDate"])
    
    # Compute min and max game dates
    min_game_date = df["gameDate"].min()
    max_game_date = df["gameDate"].max()
    
    print(f"📅 Earliest Game Date: {min_game_date}")
    print(f"📅 Latest Game Date: {max_game_date}")
    
    # Define the requested API date (replace "now" with actual date used in request)
    api_requested_date = datetime.datetime.today()  # If "now"
    # api_requested_date = datetime.datetime(2025, 2, 22)  # Example date used in request
    
    # Compute the difference between requested date and min/max game dates
    days_before = (api_requested_date - min_game_date).days
    days_after = (max_game_date - api_requested_date).days
    
    print(f"📉 Days Before Requested Date: {days_before} days")
    print(f"📈 Days After Requested Date: {days_after} days")

check_gameDateRangeInAPI(df_games)

In [ ]:
#lets now try to change the API response date to see if the range is the same -- 
#no results returned because the scoreboard API doesnt take date as a parameter (only accepts "now" or current date)

df_games = load_gamesFromScoreboardAPI(str("2024-12-31"))
check_gameDateRangeInAPI(df_games)

OPTION 2 for GAME DATA
Lets create second option for getting game data:
1. Use team schedule API (https://api-web.nhle.com/v1/club-schedule-season/dal/now) to get list of game (gameIDs) for each team for the current season
2. Use gamecenter API to retrieve game results for each gameID from #1

In [ ]:
# let's first create a function that loads API response in a readable dictionary so we can investigate the structure

def read_APIResponseStructure(url):
    
    # Fetch game data
    response = requests.get(url)
    data = response.json()

    pp = pprint.PrettyPrinter(depth=4)  # Limits deep nesting
    pp.pprint(data)

In [ ]:
# lets check the club-schedule-season API 

url = "https://api-web.nhle.com/v1/club-schedule-season/dal/now"

read_APIResponseStructure(url)

In [ ]:
# the club-schedule-season API has all the data we need for a new df_games that has ALL games in teh current season
# lets write a new function for loading the current season games

def load_gamesFromClubScheduleAPI():
    all_games = []  # List to store all games data
    
    # Loop through each team and fetch their games in the current season
    for index, team in df_teams.iterrows():
        team_name = team["teamName"]["default"]  # only extract the default value - it has a nested dictionary of format (default, value)
        team_abbrev = team["teamAbbrev"]["default"]
        
        print(f"Fetching games for ({team_abbrev})...")
        
        # NHL API URL for games this season for that team
        # now = current season = 20242025
        game_url = f"https://api-web.nhle.com/v1/club-schedule-season/{team_abbrev}/now"

        print(f"Fetching data from: {game_url}")
        
        try:
            #get API call - fetch game data
            response = requests.get(game_url)
            if response.status_code != 200:
                print(f"⚠️ Error: Received status code {response.status_code} from API")
                print(f"Response: {response.text}")  # Print error details
                continue  # Skip to next team if the request fails
            
            # retrieve JSON response
            data = response.json()

            # check if there's any data for this team
            if "games" not in data or not data["games"]:
                print(f"⚠️ No games found for {team_abbrev}")
                continue  # Skip to the next team

            # Loop through the JSON structure and pull out key game details (Game Date for that Team > Game > Key Fields) 
            for game in data["games"]:
                    
                    # Convert gameDate to a datetime object
                    game_date = datetime.datetime.strptime(game["gameDate"], "%Y-%m-%d").date()

                    # Only keep games that occurred today or earlier
                    if game_date < datetime.date.today():
                        all_games.append({
                            "gameID": game["id"],
                            "gameDate": game["gameDate"],
                            "season": game["season"],
                            "gameDetailURL": game["gameCenterLink"],
                            "venue": game["venue"]["default"],
                            "startTime": game["startTimeUTC"],
                            "awayTeam": game["awayTeam"]["commonName"]["default"],
                            "awayTeamAbbrev": game["awayTeam"]["abbrev"],
                            "awayTeamScore": game.get("awayTeam",{}).get("score",0), # if score doesn't exist (game in future), then return 0 (for simplicity)
                            "homeTeam": game["homeTeam"]["commonName"]["default"],
                            "homeTeamAbbrev": game["homeTeam"]["abbrev"],
                            "homeTeamScore": game.get("homeTeam",{}).get("score",0) # if score doesn't exist (game in future), then return 0 (for simplicity)
                        })
            
        except Exception as e:
            print(f"Error fetching roster for {team_name}: {e}")  #error handling on API call
    
    
    # Convert to DataFrame
    df_games = pd.DataFrame(all_games)
    
    # Display first few rows
    df_games.info()
    df_games.head()   

    return df_games

df_games = load_gamesFromClubScheduleAPI()


In [ ]:
#now we have 2067 games! Much better!
#lets check the gameDate range -- good news, it goes from Sep 9 2024 (start of season) to Feb 24 (yesterday's date)

# Compute min and max game dates
min_game_date = df_games["gameDate"].min()
max_game_date = df_games["gameDate"].max()

print(f"📅 Earliest Game Date: {min_game_date}")
print(f"📅 Latest Game Date: {max_game_date}")

In [ ]:
show_memory_usage()
optimize_dataframe(df_games)

In [ ]:
""" - skip this step as games + players merges creates N:M matrix -- 450MB data frame!!

# step 3 merge player and game data
# we will merge on team code (teamAbbrev) that is is both datasets
# we need to merge twice because in the game data, we have two teams - away (awayTeamAbbrev), and home (homeTeamAbbrev)

# Merge home team players
df_games_players = df_games.merge(
    df_players, 
    left_on="homeTeamAbbrev", 
    right_on="teamAbbrev", 
    how="left", 
    suffixes=("_game", "_homePlayer")
)

# Merge away team players
df_games_players = df_games_players.merge(
    df_players, 
    left_on="awayTeamAbbrev", 
    right_on="teamAbbrev", 
    how="left", 
    suffixes=("_home", "_away")
)

# Display merged dataset
df_games_players.head()

"""

In [ ]:
show_memory_usage()
optimize_dataframe(df_games_players)

In [ ]:
del df_games_players

In [ ]:
gc.collect()

In [ ]:
show_memory_usage()

In [ ]:
#That 445MB memory entry (_56) is likely a stored reference to an old variable (probably the deleted df_games_players or another large DataFrame).
#Even though you deleted df_games_players, Python’s interactive Jupyter environment keeps hidden references (_N variables) that store old outputs until you explicitly clear them.
%reset out -f

#This forces Jupyter to forget all _N outputs (like _56).

In [ ]:
show_memory_usage()

In [ ]:
# the df_games_players merged data frame was too large because of N:M relationship between games and players
# let's simplify to only what we need for the model
# we will pull aggregate stats from players and add them to the games data frame directly - so our data set is just for N games but added features
# added features from players to games: avg weight, avg height, nationality counts, avg age, 

#1 - Take mean of weight and height and add them to a temporary dataframe for aggregate player features by team
df_team_features = df_players.groupby("teamAbbrev").agg({
    "weightInPounds": "mean",
    "heightInInches": "mean"
}).reset_index()

# Rename columns for clarity
df_team_features.rename(columns={
    "weightInPounds": "avg_team_weight",
    "heightInInches": "avg_team_height"
}, inplace=True)

# Display the new team-level DataFrame
df_team_features.head()

In [ ]:
#2 lets convert birthdate to age and add to team features

# Ensure birthDate is in datetime format
df_players["birthDate"] = pd.to_datetime(df_players["birthDate"])

# Function to calculate age
def calculate_age(birthdate):
    today = pd.Timestamp.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

# Apply function to compute age
df_players["age"] = df_players["birthDate"].apply(calculate_age)

# Compute new feature: Average Age per Team  - **another temporary data frame, remember to delete
df_team_age = df_players.groupby("teamAbbrev").agg({
    "age": "mean"
}).reset_index().rename(columns={"age": "avg_team_age"})

# Merge new feature into df_team_features without overwriting
df_team_features = df_team_features.merge(df_team_age, on="teamAbbrev", how="left")

# Display first few rows
df_team_features.head()

In [ ]:
#3 lets add nationality composition of each team by taking a count of players by birth country
#lets start by profiling what countries are included in birthCountry field and what the syntax is

df_players["birthCountry"].value_counts()


In [ ]:
#3a lets add nationality composition of each team by taking a count of players by birth country
# now lets add a field for categorizing them - we'll use this for creating counts for each big nation by team

# Define function to categorize birth countries
def categorize_birth_country(country):
    if country in ["CAN"]:
        return "CAN"
    elif country in ["USA"]:
        return "USA"
    elif country in ["SWE"]:
        return "SWE"
    elif country in ["FIN"]:
        return "FIN"
    elif country in ["RUS"]:
        return "RUS"
    else:
        return "OTHER"

# Apply function to create a new column
df_players["birthCountryCategory"] = df_players["birthCountry"].str.upper().apply(categorize_birth_country)

# Display first few rows to verify
df_players[["lastName", "birthCountry", "birthCountryCategory"]].head()

In [ ]:
#3b we will now create a count by big country for each team and put it in a temp data frame

# Aggregate birth country counts per team 
# (1) group df_play by team and country, (2) size = count by grouping, (3) unstack = converts birthCountryCate values to columns, (4) reset index = turn team name from index to a column, create unique ID
df_team_country_counts = df_players.groupby(["teamAbbrev", "birthCountryCategory"]).size().unstack(fill_value=0).reset_index()

# Rename columns for clarity
df_team_country_counts.rename(columns={
    "CAN": "num_CAN_players",
    "USA": "num_USA_players",
    "SWE": "num_SWE_players",
    "FIN": "num_FIN_players",
    "RUS": "num_RUS_players",
    "Other": "num_OTHER_players"
}, inplace=True)

# Display first few rows
df_team_country_counts.head()

In [ ]:
#3c add the team country counts to the team_features via merge

# Merge new feature into df_team_features without overwriting
df_team_features = df_team_features.merge(df_team_country_counts, on="teamAbbrev", how="left")

# Display first few rows
df_team_features.head()

In [ ]:
#lets check df_players if there are other team summary features we want to add
df_players.info()

In [ ]:
#lets profile shootsCatches column to see what values are in there
df_players["shootsCatches"].value_counts()

In [ ]:
# lets add another summary feature for how many left or right shots each team has with the shootsCatches column

# Aggregate L and R shot counts per team 
df_shot_counts = df_players.groupby(["teamAbbrev", "shootsCatches"]).size().unstack(fill_value=0).reset_index()

# Rename columns for clarity
df_shot_counts.rename(columns={
    "L": "num_LeftShot_players",
    "R": "num_RightShot_players",
}, inplace=True)

# Display first few rows
df_shot_counts.head()

In [ ]:
# Merge new feature into df_team_features without overwriting
df_team_features = df_team_features.merge(df_shot_counts, on="teamAbbrev", how="left")

# Display first few rows
df_team_features.head()

In [ ]:
""""Dead code - # step 4 analyze, visualize and interpret  - does team size affect game outcome?

# take average of players height for home and away
df_games_players["avg_home_height"] = df_games_players.groupby("homeTeamAbbrev")["heightInInches_home"].transform("mean")
df_games_players["avg_away_height"] = df_games_players.groupby("awayTeamAbbrev")["heightInInches_away"].transform("mean")

# take average of players weight for home and away
df_games_players["avg_home_weight"] = df_games_players.groupby("homeTeamAbbrev")["weightInPounds_home"].transform("mean")
df_games_players["avg_away_weight"] = df_games_players.groupby("awayTeamAbbrev")["weightInPounds_away"].transform("mean")

df_games_players[["homeTeamAbbrev", "awayTeamAbbrev", "avg_home_height", "avg_away_height", "avg_home_weight", "avg_away_weight"]].head()
"""

In [ ]:
df_team_features.info()

In [ ]:
# merge team features into the games data frame 
#we'll have features for both the home and away teams, so we'll need to merge twice

#merge team player features with home team of games
df_games = df_games.merge(df_team_features, left_on="homeTeamAbbrev", right_on="teamAbbrev", how="left")

# Rename columns to indicate they are for the home team
df_games.rename(columns={
    "avg_team_weight": "avg_home_weight",
    "avg_team_height": "avg_home_height",
    "avg_team_age": "avg_home_age",
    "num_CAN_players": "num_CAN_home",
    "num_FIN_players": "num_FIN_home",
    "OTHER": "num_OTHER_home",
    "num_RUS_players": "num_RUS_home",
    "num_SWE_players": "num_SWE_home",
    "num_USA_players": "num_USA_home",
    "num_LeftShot_players": "num_LeftShot_home",
    "num_RightShot_players": "num_RightShot_home"
}, inplace=True)

# Drop duplicate team column
df_games.drop(columns=["teamAbbrev"], inplace=True)


In [ ]:
# do the same merge for the away team

df_games = df_games.merge(df_team_features, left_on="awayTeamAbbrev", right_on="teamAbbrev", how="left")

# Rename columns to indicate they are for the away team
df_games.rename(columns={
    "avg_team_weight": "avg_away_weight",
    "avg_team_height": "avg_away_height",
    "avg_team_age": "avg_away_age",
    "num_CAN_players": "num_CAN_away",
    "num_FIN_players": "num_FIN_away",
    "OTHER": "num_OTHER_away",
    "num_RUS_players": "num_RUS_away",
    "num_SWE_players": "num_SWE_away",
    "num_USA_players": "num_USA_away",
    "num_LeftShot_players": "num_LeftShot_away",
    "num_RightShot_players": "num_RightShot_away"
}, inplace=True)

# Drop duplicate team column
df_games.drop(columns=["teamAbbrev"], inplace=True)



In [ ]:
# check the merge worked

df_games.info()
df_games.head()

# we get 2043 games with all the fields from games + team features (player summary) - perfect! 

In [ ]:
# Determine winning team & add column 
df_games["winningTeam"] = df_games.apply(
    lambda x: x["homeTeamAbbrev"] if x["homeTeamScore"] > x["awayTeamScore"] 
    else (x["awayTeamAbbrev"] if x["homeTeamScore"] < x["awayTeamScore"] else "Tie"),
    axis=1
)

In [ ]:
# visualize height, weight and game win/losses

# Plot average player weight in winning teams (*flaw: uses the home team's weight in all cases, even when home team won)
plt.figure(figsize=(10, 5))
sns.boxplot(x=df_games["winningTeam"], y=df_games["avg_home_weight"], palette="coolwarm")
plt.title("Average Player Weight in Winning Teams")
plt.xticks(rotation=45)
plt.show()


In [ ]:
#step 4 analyze - home vs away win %

# Count home and away wins
home_wins = (df_games["winningTeam"] == df_games["homeTeamAbbrev"]).sum()
away_wins = (df_games["winningTeam"] == df_games["awayTeamAbbrev"]).sum()
ties = (df_games["winningTeam"] == "Tie").sum()

# Print results
print(f"🏠 Home Wins: {home_wins}")
print(f"✈️ Away Wins: {away_wins}")
print(f"🤝 Ties: {ties}")

In [ ]:
# plot home vs. away %

# Data for visualization
win_data = pd.DataFrame({
    "Category": ["Home Wins", "Away Wins", "Ties"],
    "Count": [home_wins, away_wins, ties]
})

# Create bar plot
plt.figure(figsize=(8, 5))
sns.barplot(x="Category", y="Count", data=win_data, palette="coolwarm")

# Add labels and title
plt.title("🏒 Home vs. Away Win Distribution")
plt.ylabel("Number of Wins")
plt.xlabel("Game Outcome")

# Show the plot
plt.show()

In [ ]:
""" Dead code - we aggregated player states per team above

# redo - games are inflated because game-player table has each game duplicated for x players on a team.  Lets fix this by removing duplicate game records

df_unique_games = df_games_players.drop_duplicates(subset=["gameID"])

# Count home and away wins using unique game IDs
home_wins = (df_unique_games["winningTeam"] == df_unique_games["homeTeamAbbrev"]).sum()
away_wins = (df_unique_games["winningTeam"] == df_unique_games["awayTeamAbbrev"]).sum()
ties = (df_unique_games["winningTeam"] == "Tie").sum()

# Print results
print(f"🏠 Home Wins: {home_wins}")
print(f"✈️ Away Wins: {away_wins}")
print(f"🤝 Ties: {ties}")

total_games = home_wins + away_wins + ties

home_win_rate = (home_wins / total_games) * 100
away_win_rate = (away_wins / total_games) * 100
tie_rate = (ties / total_games) * 100

# Print win percentages
print(f"🏠 Home Win Rate: {home_win_rate:.2f}%")
print(f"✈️ Away Win Rate: {away_win_rate:.2f}%")
print(f"🤝 Tie Rate: {tie_rate:.2f}%")

import matplotlib.pyplot as plt
import seaborn as sns

# Data for visualization
win_data = pd.DataFrame({
    "Category": ["Home Wins", "Away Wins", "Ties"],
    "Count": [home_wins, away_wins, ties]
})

# Create bar plot
plt.figure(figsize=(8, 5))
sns.barplot(x="Category", y="Count", data=win_data, palette="coolwarm")

# Add labels and title
plt.title("🏒 Home vs. Away Win Distribution (Unique Games)")
plt.ylabel("Number of Wins")
plt.xlabel("Game Outcome")

# Show the plot
plt.show()

"""


In [ ]:
Train an AI Model to Predict Game Winners 🏆🔮
Now that we have structured NHL game data, we can train a machine learning model to predict which team will win based on team and player attributes.

Overview: How We’ll Build the Model

1️⃣ Define the Problem – Predict whether the home team or away team will win.
2️⃣ Select Features – Choose player & team stats as inputs.
3️⃣ Prepare the Dataset – Convert categorical data, handle missing values.
4️⃣ Train Machine Learning Models – Use Logistic Regression, Random Forest, or XGBoost.
5️⃣ Evaluate Model Performance – Check accuracy, precision, recall.
6️⃣ Make Predictions on Future Games – Predict upcoming match results.

In [ ]:
# refresh on contents of data we have

df_games.info()
df_games.head()

In [ ]:
"""
Step 1: Define the Problem

Input (X): Team/player statistics before the game.
Output (Y): winningTeam, where:
1 = Home Team Wins
0 = Away Team Wins
"""

# Create the target variable
df_games["homeWin"] = (df_games["winningTeam"] == df_games["homeTeamAbbrev"]).astype(int)

# Drop unneeded columns
df_model = df_games.drop(columns=["winningTeam", "gameID"])

# verify 
df_model.info()
df_model.head()

In [ ]:
"""
Step 2: Select Features for the Model

We need to choose numerical features that might influence game outcomes.

🔹 Key Features to Include
Team Attributes:
avg_home_weight, avg_away_weight (Avg. player weight per team)
avg_home_height, avg_away_height (Avg. player height per team)
homeTeamScore, awayTeamScore (Previous performance)
avg_away_age, avg_home_age (experience/age of players on team)
num_<country>_players (nationality of players on team)
num_LeftShot_players, num_RightShot_players (right/left handed shooters on team)
"""

# Select features
features = [
    "avg_home_weight", "avg_away_weight",
    "avg_home_height", "avg_away_height",
    "homeTeamScore", "awayTeamScore",
    "avg_home_age", "avg_away_age",
    "num_LeftShot_home", "num_RightShot_home",
    "num_LeftShot_away", "num_RightShot_away",
    "num_CAN_home", "num_CAN_away",
    "num_USA_home", "num_USA_away",
    "num_FIN_home", "num_FIN_away",
    "num_SWE_home", "num_SWE_away",
    "num_RUS_home", "num_RUS_away",
    "num_OTHER_home", "num_OTHER_away"
]

# set X (features) and Y (homeWin team wins yes/1 or no/0) in the model
X = df_model[features]
y = df_model["homeWin"]

#check parameters
X.info()
y.info()


In [ ]:
# Monitor DataFrame Memory Usage

def show_memory_usage():
    """Displays memory usage of all DataFrames in global scope."""
    for var_name in list(globals()):
        var_value = globals()[var_name]
        if isinstance(var_value, (pd.DataFrame, pd.Series)):  # Check if it's a DataFrame
            print(f"{var_name}: {sys.getsizeof(var_value) / (1024 * 1024):.2f} MB")

show_memory_usage()  # Run this after loading data

In [ ]:
# Delete Large DataFrames When No Longer Needed

def delete_dataframe(df):
    del df
    gc.collect()  # Force garbage collection


In [ ]:
# Optimize DataFrame Column Types (from 64->32 float or int vars)

def optimize_dataframe(df):
    """Reduce memory usage by downcasting numerical columns."""
    for col in df.select_dtypes(include=['float64']):
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes(include=['int64']):
        df[col] = df[col].astype('int32')
    return df


In [ ]:
# remove hidden variables that are taking up memory

def remove_hidden_variables():
    %reset out -f


In [ ]:
#clean up memory

remove_hidden_variables()
optimize_dataframe(df_model)

In [ ]:
show_memory_usage()

Step 3️⃣ Prepare the Dataset
- Fill missing values with mean
- Scale features
- Split data intro training and validation / testing sets

In [ ]:
#3a: fill missing values with mean

X.fillna(X.mean(), inplace=True)

In [ ]:
#3b: scale features that have different ranges - so that they have mean ~ 0 and std dev ~ 1, ideal for model performance

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Standardizes all numeric features


In [ ]:
# verify --> shows that are n (2043) games by 24 features (# of cols) in the array.  And first 5 features have mean ~0 and std dev ~1

import numpy as np

print(f"Shape of X_scaled: {X_scaled.shape}")  # Check dimensions
print(f"Mean of first 5 features: {np.mean(X_scaled, axis=0)[:5]}")  # Check mean
print(f"Std Dev of first 5 features: {np.std(X_scaled, axis=0)[:5]}")  # Check standard deviation


In [ ]:
#3c: split data between training set (80% of X) and validation/testing set (20% of X)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# random_state provides a seed so that results are re-producable (same sample each time)
# stratify preserves the distribution of y / homeWin in each sample (eg. 60% home wins)

In [ ]:
# verify size of training and test sets

y_test.value_counts()